In [ ]:
!pip install -q cellxgene-census
!pip install -q obonet

In [1]:
from os.path import join

import pandas as pd
import numpy as np

from tqdm.auto import tqdm

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Utils code

In [2]:
import obonet
import networkx


# url = 'http://purl.obolibrary.org/obo/cl/cl-simple.obo'
url = 'https://github.com/obophenotype/cell-ontology/releases/download/v2023-05-22/cl-simple.obo'
graph = obonet.read_obo(url, ignore_obsolete=True)

# only use "is_a" edges
edges_to_delete = []
for i, x in enumerate(graph.edges):
    if x[2] != 'is_a':
        edges_to_delete.append((x[0], x[1]))
for x in edges_to_delete:
    graph.remove_edge(u=x[0], v=x[1])

# define mapping from id to name
id_to_name = {id_: data.get('name') for id_, data in graph.nodes(data=True)}
# define inverse mapping from name to id
name_to_id = {v: k for k, v in id_to_name.items()}


def find_child_nodes(cell_type):
    return [id_to_name[node] for node in networkx.ancestors(graph, name_to_id[cell_type])]


def find_parent_nodes(cell_type):
    return [id_to_name[node] for node in networkx.descendants(graph, name_to_id[cell_type])]


# Select data to download

In [3]:
import cellxgene_census

census = cellxgene_census.open_soma(census_version="2023-05-15")

In [6]:
summary = census["census_info"]['summary']

In [4]:
PROTOCOLS = [
    "10x 5' v2", 
    "10x 3' v3", 
    "10x 3' v2", 
    "10x 5' v1", 
    "10x 3' v1", 
    "10x 3' transcription profiling", 
    "10x 5' transcription profiling"
]


COLUMN_NAMES = [
    "soma_joinid",
    "is_primary_data",
    "dataset_id", 
    "donor_id",
    "assay", 
    "cell_type", 
    "development_stage", 
    "disease", 
    "tissue", 
    "tissue_general"
]

In [5]:
obs = (
    census["census_data"]["homo_sapiens"]
    .obs
    .read(
        column_names=COLUMN_NAMES,
        value_filter=f"is_primary_data == True and assay in {PROTOCOLS}"
    )
    .concat()
    .to_pandas()
)

In [6]:
obs['tech_sample'] = (obs.dataset_id + '_' + obs.donor_id).astype('category')

for col in COLUMN_NAMES:
    if obs[col].dtype == object:
        obs[col] = obs[col].astype('category')


In [7]:
obs.dtypes

soma_joinid             int64
is_primary_data          bool
dataset_id           category
donor_id             category
assay                category
cell_type            category
development_stage    category
disease              category
tissue               category
tissue_general       category
tech_sample          category
dtype: object

In [8]:
# remove all cell types which are not a subtype of native cell
cell_types_to_remove = obs[~obs.cell_type.isin(find_child_nodes('native cell'))].cell_type.unique().tolist()

# remove all cell types which have less than 5000 cells
cell_freq = obs.cell_type.value_counts()
cell_types_to_remove += cell_freq[cell_freq < 5000].index.tolist()

# remove cell types which have less than 30 tech_samples
tech_samples_per_cell_type = obs[['cell_type', 'tech_sample']].groupby('cell_type').agg({'tech_sample': 'nunique'}).sort_values('tech_sample')
cell_types_to_remove += tech_samples_per_cell_type[tech_samples_per_cell_type.tech_sample <= 30].index.tolist()

# filter out too granular labels
# remove all cells that have <= 7 parents in the cell ontology
cell_types = obs.cell_type.unique().tolist()

n_children = []
n_parents = []

for cell_type in cell_types:
    n_parents.append(len(find_parent_nodes(cell_type)))
    n_children.append(len(find_child_nodes(cell_type)))

cell_types_to_remove += (
    pd.DataFrame({'n_children': n_children, 'n_parents': n_parents}, index=cell_types)
    .query('n_parents <= 7')
    .index.tolist()
)
cell_types_to_remove = list(set(cell_types_to_remove))

In [9]:
obs.cell_type.nunique() - len(cell_types_to_remove)

164

In [10]:
obs_subset = obs[~obs.cell_type.isin(cell_types_to_remove)].copy()
for col in obs_subset.columns:
    if obs_subset[col].dtype == 'category':
        obs_subset[col] = obs_subset[col].cat.remove_unused_categories()
obs_subset

,soma_joinid,is_primary_data,dataset_id,donor_id,assay,cell_type,development_stage,disease,tissue,tissue_general,tech_sample
15,15,True,9d8e5dca-03a3-457d-b7fb-844c75735c83,donor-GOLD,10x 3' v3,macrophage,53-year-old human stage,normal,subcutaneous abdominal adipose tissue,adipose tissue,9d8e5dca-03a3-457d-b7fb-844c75735c83_donor-GOLD
16,16,True,9d8e5dca-03a3-457d-b7fb-844c75735c83,donor-GOLD,10x 3' v3,macrophage,53-year-old human stage,normal,subcutaneous abdominal adipose tissue,adipose tissue,9d8e5dca-03a3-457d-b7fb-844c75735c83_donor-GOLD
18,18,True,9d8e5dca-03a3-457d-b7fb-844c75735c83,donor-GOLD,10x 3' v3,endothelial cell,53-year-old human stage,normal,subcutaneous abdominal adipose tissue,adipose tissue,9d8e5dca-03a3-457d-b7fb-844c75735c83_donor-GOLD
19,19,True,9d8e5dca-03a3-457d-b7fb-844c75735c83,donor-GOLD,10x 3' v3,macrophage,53-year-old human stage,normal,subcutaneous abdominal adipose tissue,adipose tissue,9d8e5dca-03a3-457d-b7fb-844c75735c83_donor-GOLD
20,20,True,9d8e5dca-03a3-457d-b7fb-844c75735c83,donor-GOLD,10x 3' v3,macrophage,53-year-old human stage,normal,subcutaneous abdominal adipose tissue,adipose tissue,9d8e5dca-03a3-457d-b7fb-844c75735c83_donor-GOLD
...,...,...,...,...,...,...,...,...,...,...,...
28169969,53794723,True,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,3,10x 3' v2,pericyte,51-year-old human stage,normal,lung,lung,8c42cfd0-0b0a-46d5-910c-fc833d83c45e_3
28169970,53794724,True,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,3,10x 3' v2,pericyte,51-year-old human stage,normal,lung,lung,8c42cfd0-0b0a-46d5-910c-fc833d83c45e_3
28169971,53794725,True,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,3,10x 3' v2,pericyte,51-year-old human stage,normal,lung,lung,8c42cfd0-0b0a-46d5-910c-fc833d83c45e_3
28169972,53794726,True,8c42cfd0-0b0a-46d5-910c-fc833d83c45e,3,10x 3' v2,pericyte,51-year-old human stage,normal,lung,lung,8c42cfd0-0b0a-46d5-910c-fc833d83c45e_3


In [11]:
cell_types_to_keep = obs_subset.cell_type.unique().tolist()

# Download data

In [12]:
protein_coding_genes = pd.read_parquet('features.parquet').gene_names.tolist()

In [ ]:
BASE_PATH = '/mnt/dssfs02/cxg_census/h5ad_raw_2023_05_15'


# download in batches to not run out of memory
for i, idxs in tqdm(enumerate(np.array_split(obs_subset.soma_joinid.to_numpy(), 20))):
    adata = cellxgene_census.get_anndata(
        census=census,
        organism="Homo sapiens",
        X_name='raw',
        obs_coords=idxs.tolist(),
        var_value_filter=f"feature_name in {protein_coding_genes}",
        column_names={"obs": COLUMN_NAMES, "var": ['feature_id', 'feature_name']},
    )
    adata.write_h5ad(join(BASE_PATH, f'{i}.h5ad'))


19it [19:39:25, 3355.84s/it]